In [30]:

from Experiment import Experiment
import matplotlib.pyplot as plt
import Plotters 
import plotly.graph_objects as go
import plotly
import numpy as np

import time
import matplotlib.cm as colormap

%matplotlib qt


loadir = 'H:/My Drive/dark 2022/csv_dark' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis
exp_name =['manipulated_2022_02_03_dark','manipulated_2023_08_09_60ms']

# ['manipulated_2023_08_07_10ms',
#             'manipulated_2023_08_06_40ms',
#            'manipulated_2023_08_09_60ms',
#            'manipulated_2023_08_09_80ms',
#            'manipulated_2023_08_10_100ms',
#            'manipulated_2022_02_03_dark',
#            'manipulated_2022_01_31_darkdown']

def dont_show_movs(exp_name,movs_to_del):
    return [mov_name for mov_name in experiments[exp_name].mov_names if mov_name not in movs_to_del ] 

mov_dark_delete = ['mov30','mov40','mov57','mov59']
mov_100_delete = ['mov10','mov102','mov19','mov34','mov35','mov47','mov51','mov52','mov66','mov68','mov73','mov90']
mov_80_delete = ['mov119','mov49','mov97','mov99']
mov_40_delete = ['mov64']
mov_60_delete = ['mov12','mov13','mov147','mov165','mov21','mov24','mov26','mov48','mov86']
mov_10_delete = ['mov46','mov54']
mov_5_delete = ['mov44']


color_map = colormap.datad["tab10"]['listed']

experiments = {exp:Experiment(loadir,exp) for exp in exp_name} # load experiments


In [31]:
# calculations - projections, frequency, angles

[exp.from_vector_to_wing_body('Y_x_body',['vectors','body'])  for exp in experiments.values()] # smooth and derive


[exp.smooth_prop_movies('CM_real_x_body',['','_dot','_dot_dot'],'body')  for exp in experiments.values()] # smooth and derive


[exp.project_axes_movies(['X_x_body_projected','X_y_body_projected'], add_to_vectors = ['vectors','body'])  for exp in experiments.values()] # X_body axis projected to xy plane
[exp.project_axes_movies(['Y_x_body_projected','Y_y_body_projected'],axis = 'Y_x_body',add_to_vectors = ['vectors','body'])  for exp in experiments.values()] # X_body axis projected to xy plane

[exp.project_axes_movies(['CM_dot_xax','CM_dot_yax'],axis = 'CM_real_x_body_dot_smth',wing_body = 'body',add_to_vectors = ['body','vectors'])  for exp in experiments.values()] # Velocity direction in the projected X_body axes (xy plane) 

# calculate the size of the velocity projected on the xy plane
[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected',add_to_vectors = True)  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected',add_to_vectors = True)  for exp in experiments.values()]


# calculate the size of the acceleration projected on the xy plane
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected')  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected')  for exp in experiments.values()]

[exp.pqr_movies() for exp in experiments.values()] # calculate p,q,r (angular velocity in body axis)

[exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi') for exp in experiments.values()] # calculate the mean value of two properties


[exp.substruct_first_frame('yaw_body','body')  for exp in experiments.values()] # substract the zero frame from a property ( {prop}_min_ref_frame )

# calculate the difference between current direction and another one at the same time:
# the angle between Xbody on the xy plane and the velocity direction on the xy plane 
[exp.delta_ang_all_time_movies('X_x_body_projected','CM_dot_xax','slip_angle_beta',three_col = 2) for exp in experiments.values()] 

# calculate the difference in angle between current property and in referance frame:
# the angle between Xbody on the xy (in 0 time) plane and Xbody on xy plane 
[exp.delta_ref_ang_movies('X_x_body_projected','delta_ang_Xax',axis = 'X_x_body_projected')  for exp in experiments.values()]

# the angle between Xbody on the xy (in 0 time) plane and the velocity direction on the xy plane 
[exp.delta_ref_ang_movies('CM_dot_xax','delta_ang_CM_dot_x',axis = 'CM_dot_xax',delta_ang = 0) for exp in experiments.values()]

# project velocity and accelerations
[exp.project_prop_all_axes_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected')  for exp in experiments.values()]
[exp.project_prop_all_axes_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected')  for exp in experiments.values()]

[exp.project_prop_all_axes_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected')  for exp in experiments.values()]
[exp.project_prop_all_axes_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected')  for exp in experiments.values()]

[exp.project_prop_all_axes_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y_body',three_col = 2,ax_to_proj = 'Y_x_body')  for exp in experiments.values()]

# calculate amplitude of phi and frequency
[exp.calculate_phi_amp_movies('rw','mean_body')  for exp in experiments.values()]
[exp.calculate_phi_amp_movies('lw','mean_body')  for exp in experiments.values()]

[exp.calculate_freq_movies('phi_rw_min_idx', 'mean_body')  for exp in experiments.values()]
[exp.calculate_freq_movies('phi_lw_min_idx', 'mean_body')  for exp in experiments.values()]


# averge properties by stroke 
props_to_mean = ['x','y','z']
[[exp.mean_by_stroke_movies(f'CM_real_{x}_body_dot_dot_smth','mean_body','body')  for exp in experiments.values()] for x in props_to_mean]
[[exp.mean_by_stroke_movies(f'CM_dot_dot_{x}_projected_all_axes','mean_body','body')  for exp in experiments.values()] for x in ['x','y']]
[[exp.mean_by_stroke_movies(f'CM_dot_x_projected','mean_body','body')  for exp in experiments.values()] for x in props_to_mean]

[[exp.mean_by_stroke_movies(f'X_{x}_body_projected','mean_body','body')  for exp in experiments.values()] for x in ['x','y']]
[[exp.mean_by_stroke_movies(f'Y_{x}_body_projected','mean_body','body')  for exp in experiments.values()] for x in ['x','y']]


# histogram calculations
exp = experiments[list(experiments.keys())[0]]
[exp.get_peaks_movies('q','max') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_y_frame','max') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_body','max') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_x_projected','zero_v') for exp in experiments.values()]
[exp.get_peaks_movies('delta_ang_Xax','max') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_dot_x_projected','peaks_max',t0 = 5, t1 = 100) for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_x_projected','half_v') for exp in experiments.values()]

[exp.subtract_interest_time_from_time('CM_dot_x_projected_zero_v','time_m_zero_v') for exp in experiments.values()]


c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:124: RuntimeWarning:

invalid value encountered in arccos

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:369: RuntimeWarning:

divide by zero encountered in divide



[None, None]

PLOTS

In [32]:
# plot pitch and roll vs acceleration
fig = go.Figure()
exp = experiments['manipulated_2022_02_03_dark']
exp.plot_prop_movies('pitch_body','body',fig,case = 'plot_mov',prop_x = 'CM_dot_dot_x_projected_all_axes' ,add_horizontal_line=None,name = 'calculated')
fig.show()

fig = go.Figure()
exp.plot_prop_movies('roll_body','body',fig,case = 'plot_mov',prop_x = 'CM_dot_dot_y_projected_all_axes' ,add_horizontal_line=None,name = 'calculated')
fig.show()

In [44]:
# plot slip angle and delta yaw

exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('slip_angle_beta','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()

fig = go.Figure()
exp.plot_prop_movies('delta_ang_Xax','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()

In [33]:
# plot body angles
exp = experiments['manipulated_2023_08_09_60ms']
fig = go.Figure()
exp.plot_prop_movies('roll_body','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()

fig = go.Figure()
exp.plot_prop_movies('pitch_body','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()

fig = go.Figure()
exp.plot_prop_movies('yaw_body','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()


In [47]:
# velocity and acceleration

fig = go.Figure()
exp.plot_prop_movies('CM_dot_x_projected','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()

fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_x_projected','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()


fig = go.Figure()
exp.plot_prop_movies('CM_real_z_body_dot_smth','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()

In [49]:
exp = experiments['manipulated_2023_08_09_60ms']
fig = go.Figure()
exp.plot_prop_movies('CM_real_z_body_dot_smth','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()

In [35]:
# plot wings angle

phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)

fig = go.Figure()
exp.plot_prop_movies('theta_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
exp.plot_prop_movies('theta_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)

exp.plot_prop_movies('psi_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
exp.plot_prop_movies('psi_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)

exp.plot_prop_movies('phi_lw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_l_color )
exp.plot_prop_movies('phi_rw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_r_color)

fig.show()



In [36]:
exp = experiments['manipulated_2023_08_09_60ms']
mov = ['mov101']
exp.plot_3d_traj_movies('CM_dot_x_projected',mov = mov)
fig.show()



In [37]:
time_vec = [10,60,100,220,280]

xbins = dict(start=0,end=360,size = 10)
fig = Plotters.subplot_histograms_delta_prop(time_vec,experiments,'delta_ang_CM_dot_x',color_map,xbins,'body')
fig.show()


In [38]:

t_vec = [0,20,75,150,200]
delta_t = 73*5/16000*1000
prop = 'freq_phi_rw_min_idx'
data = [exp.mean_prop_time_vector_movies(prop,delta_t,t_vec, mov = False) for exp in  experiments.values()]


fig = go.Figure()
[Plotters.vplot(dat,fig,'t0','freq_phi_rw_min_idx',exp_name) for dat,exp_name in zip(data,experiments.keys())]
fig.show()


prop = 'CM_dot_x_projected'
data = [exp.mean_prop_time_vector_movies(prop,delta_t,t_vec, mov = False,sub_mean = False) for exp in  experiments.values()]


fig = go.Figure()
[Plotters.vplot(dat,fig,'t0','CM_dot_x_projected',exp_name) for dat,exp_name in zip(data,experiments.keys())]
fig.show()

plotly.offline.plot(fig, filename=f'I:/My Drive/Research/Dark/article/figures/html/CM_dot_x_violin_plot.html',auto_open=False)



c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:345: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:345: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:345: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:345: RuntimeWarning:

Mean of empty slice



'I:/My Drive/Research/Dark/article/figures/html/CM_dot_x_violin_plot.html'

In [39]:

point_name = 'CM_dot_x_projected_zero_v'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])

fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_dot_x_projected_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_dot_x_projected_peaks_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()
